In [240]:
import time
import sys
# from graph import graph # my own code for graph object
import heapq
from collections import deque,defaultdict
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable # for linear programming

In [616]:
from collections import deque,defaultdict
class vertex:
    def __init__(self,node):
        self.id = node
        self.adjacent = set()

    def __str__(self):
        # for print out result
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor):
    
        self.adjacent.add(neighbor)

    def remove_neighbor(self, neighbor):
        if neighbor in self.adjacent:
            self.adjacent.remove(neighbor)

    def is_connected(self,neighbor):
        return neighbor in self.adjacent

    def get_connections(self):
        return self.adjacent



class graph:
    # unweighted undirected graph
    # can be connected or not
    def __init__(self):
        self.vert_dict = {} # vertex_id (int) : vertex
        self.num_vertices = 0
        self.num_edges = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self,node):
        self.num_vertices += 1
        new_vertex = vertex(node)
        self.vert_dict[node] = new_vertex

    def get_vertex(self,node):
        if node in self.vert_dict:
            return self.vert_dict[node]
        else:
            return None

    def add_edge(self, frm, to):
        # for new vertices
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)

        if not self.vert_dict[frm].is_connected(self.vert_dict[to]):
            self.num_edges += 1

        self.vert_dict[frm].add_neighbor(self.vert_dict[to])
        self.vert_dict[to].add_neighbor(self.vert_dict[frm])


    def remove_edge(self, frm, to):
        self.vert_dict[frm].remove_neighbor(self.vert_dict[to])
        self.vert_dict[to].remove_neighbor(self.vert_dict[frm])
        self.num_edges -= 1
    
    def remove_vertex(self,node,inplace = False):
        
        G = self.copy() if not inplace else self
        
        for neighbor in list(G.vert_dict[node].get_connections()):
            G.remove_edge(node, neighbor.id)
            if not neighbor.get_connections():
                del G.vert_dict[neighbor.id]
        del G.vert_dict[node]
        
        return G
            

    def get_vertices(self):
        # return a list of ints, the id of vertices
        return list(self.vert_dict.keys())
    
    def get_vertex_degree(self,node):
        return len(self.get_vertex(node).adjacent)
    
    def get_max_degree_vertex(self):
        return max(self.vert_dict, key = lambda x: len(self.get_vertex(x).get_connections()))
    
    def copy(self):
        copy = graph()
        for v in self.vert_dict:
            for u in self.vert_dict[v].get_connections():
                copy.add_edge(v,u.id)
        return copy


            



In [283]:
G.get_max_degree_vertex()

2

In [382]:
# read data and construct a graph object
def parse_edges(filename):
    # parse edges from graph file to create your graph object
    # filename: string of the filename
    f = open(filename, "r")
    n_vertices, n_edges, _ = f.readline().split(' ')
    n_vertices, n_edges = int(n_vertices), int(n_edges)

    G = graph() # create a graph

    # add edges to the graph
    for i in range(1,n_vertices+1):
        newline = f.readline()
        newline = newline.rstrip('\n')
        newline = newline.strip()
        
        if not newline:
            G.add_vertex(i)
        else:
            neighbors = list(map(int, newline.split(' ')))

            for neighbor in neighbors:
                if neighbor > i:
                    G.add_edge(i, neighbor)
                
    return G  

In [617]:
# G = parse_edges('../DATA/karate.graph')
# G = parse_edges('../DATA/netscience.graph')
G = parse_edges('../DATA/dummy2.graph')

In [626]:
for v in G:
    print(v)

1 adjacent: [2, 3]
2 adjacent: [4, 1, 3]
3 adjacent: [2, 1, 5]
4 adjacent: [2, 6, 5]
5 adjacent: [4, 3]
6 adjacent: [4]


In [619]:
G1 = G.remove_vertex(4)

## 2-approximation Algorithm to find a lower bound

In [210]:
def approx2(G):
    # G: a graph object
    # return: the number of vertices of a 2-approximation solution to the min vertex cover problem of G
    S = set()
    for v in G:
        if v in S:
            continue
        for neighbor in v.get_connections():
            
            if neighbor.id < v.id:
                continue
            if v.id not in S and neighbor.id not in S:
                S.add(v.id)
                S.add(neighbor.id)
    return len(S)

Since $|S| \leq 2 OPT(G)$, we know $|S|/2$ is a lower bound of $OPT(G)$.

In [227]:
LB = approx2(G) // 2
n = G.num_vertices

In [228]:
LB

94

## Branch-and-Bound

$C'$: partial vertex cover of $G = (V, E)$

$G' = (V', E')$: subgraph not covered by $C'$

$V' = V - C' - \{u | \forall (u,v) \in E,  v \in C'\}$

$E' = \{(u,v) \in E | u, v \in V' \}$

LB = $|C'|$ + LB of the VC for $G'$

* If we have $V'$ empty, we found a vertex cover
* LB of the VC for $G'$ can be found using approx2($G'$)
* The problem is how to do recursion and how to find and update $G'$

In [464]:
def Find_Vprime(G, Cprime):
    'Cprime: a list of numbers in {1, 2, ..., n}'
    'Return: a set of vertices Vprime of the graph Gprime'
    Vprime = set()
#     Cprime = set(Cprime)
    n = G.num_vertices
    for node in range(1, n+1):
        if node in Cprime:
            continue
            
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id not in Cprime:
                Vprime.add(node)
                break
    return Vprime

In [430]:
def Find_LowerBound(V_prime):
    'Find lower bound by linear programming, slow'
    'Cprime: a list of vertices of a vertex cover'
   
    V_prime = {i:x for i,x in zip(V_prime, range(len(V_prime)))} # nodes: variable index
    
    # Prepare for Linear Regression
    obj = [1] * len(V_prime) # coeff of objective function
    bnd = [(0,1)] * len(V_prime) # bounds for each variable

    # Linear constraints, has to be <= 
    lhs_ineq = []
    rhs_ineq = []

    for node in V_prime:
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id > node and neighbor.id in V_prime:
                new_constraint = [0]*len(V_prime)
                new_constraint[V_prime[node]] = -1
                new_constraint[V_prime[neighbor.id]] = -1
                lhs_ineq.append(new_constraint)
                rhs_ineq.append(-1)

    # solve the linear programming problem
    print('Number of constraints:', len(lhs_ineq))
    opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd,
                  method="revised simplex")
    
    # lowerbound is half of the optimal of the linear optimization problem
    lowerbound = opt.fun // 2
    
    return lowerbound

In [435]:
def Gprime(V_prime):
    Gprime = graph()
    for node in V_prime:
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id > node and neighbor.id in V_prime:
                Gprime.add_edge(node,neighbor.id)
    return Gprime

In [654]:
G = parse_edges('../DATA/netscience.graph')

In [659]:
## This code is working, but not very efficient, trying to improve the speed using the following cell
class node:
    def __init__(self,id, parent,lb,state):
        self.id = id
        self.parent = parent
        self.lb = LB
        self.state = state
#         self.degree = degree
    
    def __lt__(self, other):
        return self.lb > other.lb
        
# dummy = node(0,None,LB)       

n = G.num_vertices
opt_num = n
vertices = G.get_vertices()
vertices.sort(key = lambda x: -len(G.get_vertex(x).adjacent))
opt_cover = vertices
LB = approx2(G) / 2
vertices_order = {vertex: i for i, vertex in enumerate(vertices)}

num_uncov_edges0 = G.num_edges
num_uncov_edges1 = G.num_edges - G.get_vertex_degree(vertices[0])
pqueue = [(num_uncov_edges0, node(vertices[0],None,LB,0)),(num_uncov_edges1,node(vertices[0],None,LB,1))]
heapq.heapify(pqueue)

while pqueue:

    num_uncov_edges,Dnode = heapq.heappop(pqueue)
    parent = Dnode.parent
    node_id = Dnode.id   
        
        
    Cprime = set([Dnode.id]) if Dnode.state else set()
    explored = set() if Dnode.state else set([Dnode.id])
    while parent:
#         print([parent.id, parent.state],end='')
        if parent.state:
            Cprime.add(parent.id)
        else:
            explored.add(parent.id)
        parent = parent.parent
#     print('')
#     print('Cprime',Cprime) 
    V_prime = Find_Vprime(G,Cprime)
    G_prime = Gprime(V_prime)
    
    new_node_id = find_next(V_prime, G_prime, explored)
    
    if not new_node_id:
        continue

#     new_node_id = vertices[vertices_order[node_id]+1]
#         print(new_node_id)
    cover_size = len(Cprime) + 1  
    new_cover = Cprime.union(set([new_node_id]))
#         print(new_cover)
    # get Vprime
#         print('new cover',new_cover)
    G_prime = G_prime.remove_vertex(new_node_id,inplace = True)
    V_prime = G_prime.get_vertices()
#         print('V prime',V_prime)
    if not V_prime:
        # solution found
        if cover_size < opt_num:
            opt_num = cover_size
            opt_cover = new_cover 
            print('Optimal:', opt_num)
            continue

    
    LowerBound = cover_size + approx2(G_prime)//2

    if LowerBound < opt_num:
        new_node1 = node(new_node_id, Dnode, LowerBound,1)
        heapq.heappush(pqueue, (G_prime.num_edges, new_node1))
#         print('new decision node added')
#     else:
#         print('new decision node pruned')


    if Dnode.lb < opt_num:
        new_node0 = node(new_node_id, Dnode, Dnode.lb,0)
        heapq.heappush(pqueue, (num_uncov_edges, new_node0))
#         print('new decision node added')
#     else:
#         print('new decision node pruned')

        

Optimal: 2277


KeyboardInterrupt: 

In [666]:
G = parse_edges('../DATA/karate.graph')

In [667]:
approx2(G)

22

In [668]:
# idea: reuse the graphs and sets from the previous result if the current node is a child of the previous node
class node:
    def __init__(self,id, parent,lb,state):
        self.id = id
        self.parent = parent
        self.lb = LB
        self.state = state
#         self.degree = degree
    
    def __lt__(self, other):
        return self.lb > other.lb
        
# dummy = node(0,None,LB)       

n = G.num_vertices
opt_num = n
vertices = G.get_vertices()
vertices.sort(key = lambda x: -len(G.get_vertex(x).adjacent))
opt_cover = vertices
LB = approx2(G) / 2
vertices_order = {vertex: i for i, vertex in enumerate(vertices)}

num_uncov_edges0 = G.num_edges
num_uncov_edges1 = G.num_edges - G.get_vertex_degree(vertices[0])
pqueue = [(num_uncov_edges0, node(vertices[0],None,LB,0)),(num_uncov_edges1,node(vertices[0],None,LB,1))]
heapq.heapify(pqueue)

old_Dnode = None

while pqueue:

    num_uncov_edges,Dnode = heapq.heappop(pqueue)
    parent = Dnode.parent
    node_id = Dnode.id   
    # get Cprime
#     print('Now considering vertex cover:')
#     print([Dnode.id, Dnode.state],end = '')

    if Dnode.parent and Dnode.parent == old_Dnode:
#         print('child found')
        if Dnode.state:
#             Cprime = Cprime.union(set([node_id]))
#             explored = explored            
            G_prime = G_prime1
            V_prime = G_prime1
           
        else:
            Cprime.remove(node_id)
            explored.add(node_id)
#             G_prime = old_G_prime
#             V_prime = old_V_prime
            
    else:
                
        Cprime = set([Dnode.id]) if Dnode.state else set()
        explored = set() if Dnode.state else set([Dnode.id])
        while parent:
    #         print([parent.id, parent.state],end='')
            if parent.state:
                Cprime.add(parent.id)
            else:
                explored.add(parent.id)
            parent = parent.parent
    #     print('')
    #     print('Cprime',Cprime) 
        V_prime = Find_Vprime(G,Cprime)
        G_prime = Gprime(V_prime)
    
#     if not V_prime:
#         continue 
#     print(V_prime)
    print(G_prime.get_vertices())
    new_node_id = find_next(V_prime, G_prime, explored)
    
    if not new_node_id:
        continue

#     new_node_id = vertices[vertices_order[node_id]+1]
#         print(new_node_id)
    cover_size = len(Cprime) + 1  
#     new_cover = Cprime.union(set([new_node_id]))
    Cprime.add(new_node_id)
#         print(new_cover)
    # get Vprime
#         print('new cover',new_cover)
    G_prime1 = G_prime.remove_vertex(new_node_id)
    V_prime1 = G_prime1.get_vertices()
#         print('V prime',V_prime)
    if not V_prime1:
        # solution found
        if cover_size < opt_num:
            opt_num = cover_size
            opt_cover = Cprime
            print('Optimal:', opt_num)
            continue

    
    LowerBound = cover_size + approx2(G_prime1) / 2

    if LowerBound < opt_num:
        new_node1 = node(new_node_id, Dnode, LowerBound,1)
        heapq.heappush(pqueue, (G_prime1.num_edges, new_node1))
#         print('new decision node added')
#     else:
#         print('new decision node pruned')


    if Dnode.lb < opt_num:
        new_node0 = node(new_node_id, Dnode, Dnode.lb,0)
        heapq.heappush(pqueue, (num_uncov_edges, new_node0))
#         print('new decision node added')
#     else:
#         print('new decision node pruned')

#     old_Cprime = Cprime
#     old_explored = explored
#     old_G_prime = G_prime
#     old_V_prime = V_prime
    old_Dnode = Dnode

        

[1, 18, 32, 13, 9, 3, 20, 2, 12, 8, 5, 4, 6, 22, 11, 7, 14, 31, 10, 28, 29, 33, 17, 15, 16, 19, 21, 23, 24, 26, 30, 25, 27]
[8, 32, 6, 20, 4, 7, 14, 2, 3, 22, 9, 13, 11, 5, 18, 29, 33, 26, 25, 31, 10, 28, 17, 24, 19, 30, 23, 21, 15, 16, 27]


AttributeError: 'NoneType' object has no attribute 'adjacent'

In [581]:
def find_next(V_prime, G_prime, Explored):
    for v in sorted(V_prime, key = lambda x: - G_prime.get_vertex_degree(x)):
        if v not in Explored:
            return v
        

In [559]:
class node:
    def __init__(self,id, parent,lb):
        self.id = id
        self.parent = parent
        self.lb = lb
#         self.state = state
    
    def __lt__(self, other):
        return self.lb < other.lb
        
# dummy = node(0,None,LB)       

n = G.num_vertices
opt_num = n
vertices = G.get_vertices()
vertices.sort(key = lambda x: -len(G.get_vertex(x).adjacent))
opt_cover = vertices
# n = 15
# vertices = [34,1,33,2,3,4,6,32,24,5,25,7,9,30,8]
vertices_order = {vertex: i for vertex, i in zip(vertices,range(n))}
pqueue = [(n-1, vertices_order[vertex_id], node(vertex_id,None,LB)) for vertex_id in vertices]
heapq.heapify(pqueue)
while pqueue:
    _, _, Dnode = heapq.heappop(pqueue)
#     print('vertex_id:', Dnode.id)
    parent = Dnode.parent
    node_id = Dnode.id   
    # get Cprime
    Cprime = set([Dnode.id]) #if parent else []
    while parent:
        Cprime.add(parent.id)
        parent = parent.parent
#     print('Cprime',Cprime) 
#     print('pqueue', [item[2].id for item in pqueue])
    for i in range(vertices_order[node_id]+1, n): 
#         print(i)
        new_node_id = vertices[i]
#         print(new_node_id)
        cover_size = len(Cprime) + 1  
        new_cover = Cprime.union(set([new_node_id]))
#         print(new_cover)
        # get Vprime
#         print('new cover',new_cover)
        V_prime = Find_Vprime(G,new_cover)
#         print('V prime',V_prime)
        if not V_prime:
            # solution found
            if cover_size < opt_num:
                opt_num = cover_size
                opt_cover = new_cover 
                print('Optimal:', opt_num, opt_cover)
                continue
        else:
            
            G_prime = Gprime(V_prime)
            LowerBound = cover_size + approx2(G_prime)//2
            
            if LowerBound < opt_num:
                new_node = node(new_node_id, Dnode, LowerBound)
                heapq.heappush(pqueue, (n-cover_size, i, new_node))
#                 print('new covered added:', new_cover)
#                 print(new_node.id, 'added as a child of ', node_id)
#             else:
#                 print('new cover pruned:', new_cover)

#                 print('prune new cover with cover size =', cover_size, 'lower bound = ', LowerBound,'optimal conver cover size:', opt_num)
                        

Optimal: 18 {1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 24, 26, 28, 30, 31, 32, 33, 34}
Optimal: 17 {1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 24, 25, 30, 31, 32, 33, 34}
Optimal: 16 {32, 33, 2, 3, 4, 5, 6, 7, 8, 9, 1, 34, 14, 24, 25, 30}
Optimal: 15 {32, 33, 2, 3, 4, 5, 6, 7, 1, 9, 34, 14, 24, 25, 30}
Optimal: 14 {32, 33, 2, 3, 4, 5, 6, 7, 1, 9, 34, 24, 25, 30}


KeyboardInterrupt: 

In [517]:
Cprime = set([34,1,33,2,3,4,6,32,24,5,25,7,9,30])
sorted(Cprime)

[1, 2, 3, 4, 5, 6, 7, 9, 24, 25, 30, 32, 33, 34]

In [507]:
V_prime = Find_Vprime(G,Cprime)
G_prime = Gprime(V_prime)

In [510]:
LowerBound = len(Cprime) + approx2(G_prime)
LowerBound

14

In [422]:
G = parse_edges('../DATA/jazz.graph')
n = G.num_vertices
vertices = G.get_vertices()
opt_num = n
opt_cover = vertices
def Branch_and_Bound(cover_set,Gprime,explored,opt_num,opt_cover):
    
    v = choose_vertex(Gprime,explored)
    print('explored',explored)
    if not v:
#         print('no v found')
        return
    v_neighbors = Gprime.get_vertex(v).get_connections()

    Gprime.remove_vertex(v)
    cover_set.add(v)
    
    if not Gprime.get_vertices():
        # solution found
        print('solution found')
        if len(cover_set) < opt_num:
            opt_cover = cover_set
            opt_num = len(cover_set)
            print('optimal:', opt_num)
            
    lower_bound = len(cover_set) + approx2(Gprime) / 2
    if lower_bound < opt_num:
        print('get here')
        Branch_and_Bound(cover_set, Gprime, explored,opt_num,opt_cover)
    
    print('v_neighbors:', v_neighbors)
    for vertex in v_neighbors:
        Gprime.add_edge(v,vertex.id)
        print('graph added vertices')
        
    cover_set.remove(v)
#     explored.add(v)
    lower_bound = len(cover_set) + approx2(Gprime) / 2
    if lower_bound < opt_num:
        Branch_and_Bound(cover_set, Gprime, explored + [v],opt_num,opt_cover)
    
#     return opt_num, opt_cover
    

In [423]:
def choose_vertex(G,explored):
    for vertex in sorted(G.get_vertices(), key = lambda x: -len(G.get_vertex(x).get_connections())):
        if vertex not in explored:
            return vertex

In [424]:
Branch_and_Bound(set(),G,[],opt_num,opt_cover)

explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
get here
explored []
g